# Word2Vec for Tamil words. 

Converting words to vector representation and similarity map generation

Fine plural form of words


In [1]:
import pandas as pd
import numpy as np 
import re
from matplotlib import pyplot as plt
import regex
from tqdm import tqdm

In [2]:
import sys

In [3]:
sys.setrecursionlimit(2500)

In [4]:
In_Colab = False
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    In_Colab = True

In [5]:
if In_Colab:
    from google.colab import drive
    drive.mount('/gdrive')

    %cd /gdrive/My\ Drive/Tamizh/data/wiki_data

In [6]:
from constants import *

In [7]:
data = pd.read_csv("../data/tawiki_data_df.csv")

In [8]:
# Shape of the data
data.shape

(235997, 1)

In [9]:
data = data.dropna()

In [10]:
data.shape

(201665, 1)

#### Convert each sentence to list of tokens. 

*observation : this have more then 2 crore vocabulary 
* Need to find the root words  - 

In [11]:
data.head()

,Content
0,விக்கிப்பீடியா மொழிகள்
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...


### Drop NA values

### Convert as one long sentence. This will be useful for cleaning vallina migu sorrkal.

## Data Cleaning 

In [12]:
def split_text(text):
    return regex.findall(r'\X', text, regex.U)

def conv_base(text):
    return [split_map.get(k) for k in split_text(text)]

def _lastmap(text):
    if len(text[-1])>1:
        return text[-1][-1]
    return text[-1][0]

def _firstmap(text):
    return text[0][0]

def is_vallmigu_pair(text1,text2):
    try:
        text1 = conv_base(text1)
        text2 = conv_base(text2)
        if (_lastmap(text1) in vall_migu) and (_firstmap(text2) in vall_migu):
            return True
        else:
            return False
    except:
        return False

In [18]:
def clean_split(text):
    text = re.sub("\s+"," ",text.strip())
    return text.split(' ')

In [19]:
data['processed'] = data.Content.apply(clean_split)


In [20]:
data.head()

,Content,processed
0,விக்கிப்பீடியா மொழிகள்,"[விக்கிப்பீடியா, மொழிகள்]"
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...,"[இங்கு, தமிழ், விக்கிப்பீடியாவைப், பற்றிய, உங்..."
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...,"[விக்கிப்பீடியா, அதன், வாசகர்களால், ஒருமித்து,..."
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...,"[விக்கிப்பீடியாவுக்கு, கட்டுரை, எழுதி, மேம்படு..."
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...,"[முகலாயக், கட்டிடக்கலைமுகலாய, கட்டடக்கலையின், ..."


In [21]:
vallmigu_pair_set = set()

In [24]:

for row in tqdm(range(data.shape[0])):
    text = data.processed.iloc[row]
    for i in range(len(text)-2):
        if is_vallmigu_pair(text[i],text[i+1]):
            vallmigu_pair_set.add((text[i],text[i+1]))
            text[i] = ''.join(split_text(text[i])[:-1])
    data.processed.iloc[row]= text



100%|██████████| 201665/201665 [29:30<00:00, 113.88it/s]


In [25]:
data.head()

,Content,processed
0,விக்கிப்பீடியா மொழிகள்,"[விக்கிப்பீடியா, மொழிகள்]"
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...,"[இங்கு, தமிழ், விக்கிப்பீடியாவை, பற்றிய, உங்கள..."
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...,"[விக்கிப்பீடியா, அதன், வாசகர்களால், ஒருமித்து,..."
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...,"[விக்கிப்பீடியாவுக்கு, கட்டுரை, எழுதி, மேம்படு..."
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...,"[முகலாய, கட்டிடக்கலைமுகலாய, கட்டடக்கலையின், சி..."


In [26]:
len(vallmigu_pair_set)

743967

#Word2Vec

Generate word2vec representation for tokens in tamil. 



In [27]:
import pickle

with open('../data/vallmigu_pair.pkl','wb') as f:
    pickle.dump(vallmigu_pair_set,f)

In [28]:
#Generate word 2 vec

from gensim.models import Word2Vec

In [31]:
texts = data.processed.values

In [32]:
 
model = Word2Vec(texts, size=100, window=50, min_count=10, workers=6)

In [33]:
len(model.wv.vocab)

142721

In [40]:
model.wv.most_similar("பிழைகள்")

[('பிழைகளை', 0.8700655102729797),
 ('பிழையை', 0.8099845051765442),
 ('தவறுகள்', 0.8098531365394592),
 ('பிழை', 0.7952994704246521),
 ('வழுக்கள்', 0.7758928537368774),
 ('தவியில்', 0.7736037373542786),
 ('பிழையான', 0.768975019454956),
 ('உமாபதிஉமாபதி', 0.7435733079910278),
 ('பயனர்கோபிகோபி', 0.7382937073707581),
 ('கனக்ஸ்', 0.7365349531173706)]

In [ ]:
model.save("tawiki_w2v")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
